# 解析xml地图文件

In [12]:
import xml.dom.minidom

domobj = xml.dom.minidom.parse('map.xml')
elementobj=domobj.documentElement

nodeobj=elementobj.getElementsByTagName('node')
node_dict={}
for i in range(len(nodeobj)):
    node_dict[nodeobj[i].getAttribute('id')]=(nodeobj[i].getAttribute('lon'),nodeobj[i].getAttribute('lat'))
    
def getValueByKey(obj,k):
    for tag in obj:
        if tag.getAttribute('k')==k:
            return tag.getAttribute('v')
    return ''

wayobj=elementobj.getElementsByTagName('way')
way_list=[]
'''
way_types:['', 'tertiary', 'residential', 'primary', 'primary_link', 'trunk_link', 'trunk', 'tertiary_link', 'secondary', 'service', 'motorway', 'unclassified', 'cycleway', 'footway', 'motorway_link', 'pedestrian', 'secondary_link', 'track', 'steps', 'path', 'construction', 'bus_guideway', 'living_street', 'platform', 'road', 'bridleway', 'proposed', 'bus_stop', 'planned', 'corridor']
'''
for i in range(len(wayobj)):
    tagobj=wayobj[i].getElementsByTagName('tag')
    way_type=getValueByKey(tagobj,'highway')
    if way_type in ['primary','primary_link','trunk','trunk_link','secondary','secondary_link','tertiary','tertiary_link','motorway','motorway_link','unclassified','residential']:
        ndobj=wayobj[i].getElementsByTagName('nd')
        ndids=[nd.getAttribute('ref') for nd in ndobj]
        name=getValueByKey(tagobj,'name')
        if name!='':
            way_list.append((name,ndids))

# 转换open street gps坐标到高德加密坐标

In [13]:
import requests
import json
import threading
import queue
from fake_useragent import UserAgent
from progressbar import ProgressBar,Percentage,Bar,Timer,ETA,FileTransferSpeed
widgets = [
    'Progress: ',
    Percentage(), ' ',
    Bar('#'), ' ',
    Timer(), ' ',
    ETA(), ' ',
    FileTransferSpeed()
]
bar = ProgressBar(widgets=widgets, maxval=len(node_dict)).start()
q=queue.Queue()
thread_list=[]
def convert_task(ndids):
    gps_string=''
    for ndid in ndids:
        gps_string+=node_dict[ndid][0]+','+node_dict[ndid][1]+'|'
    server_host='https://restapi.amap.com/v3/assistant/coordinate/convert'
    parameters={
        'key':'053d8830d4ed8bfc7fac9a3a5d8b10cf',
        'locations':gps_string,
        'coordsys':'gps'
    }
    response=requests.get(url=server_host,
                 headers={'User-Agent':UserAgent(path='fakeuseragent.json').random},
                 params=parameters)
    response.encoding=response.apparent_encoding
    res=json.loads(response.text)
    coords=res['locations'].split(';')
    for i,ndid in enumerate(ndids):
        q.put((ndid,tuple(coords[i].split(','))))
node_dict_amap={}
for i in range(0,len(node_dict),40):
    iterate_nodes=list(node_dict.keys())[i:i+40]
    thread=threading.Thread(target=convert_task, args=(iterate_nodes,))
    thread_list.append(thread)
    thread.start()
    bar.update(i+1)
bar.finish()
for thread in thread_list:
    thread.join()
thread_list.clear()
while not q.empty():
    item=q.get()
    node_dict_amap[item[0]]=item[1]

KeyboardInterrupt: 

# 保存数据

In [14]:
import pickle
with open('map_nodes.pkl','wb') as f:
    pickle.dump(node_dict, f)
with open('map_nodes_amap.pkl','wb') as f:
    pickle.dump(node_dict_amap, f)
with open('map_ways.pkl','wb') as f:
    pickle.dump(way_list, f)